In [66]:
import pandas as pd
import numpy as np
from sklearn.metrics import euclidean_distances, recall_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split

In [67]:
df = pd.read_csv("train_dataset_train.csv")
metro = pd.read_csv("metro.csv")
kk = df.drop_duplicates(subset=['station_id'], inplace=False)

In [68]:
#сначала разберемся с датами
df.pass_dttm = pd.to_datetime(df.pass_dttm)

def hour_period(hour):
    if hour < 10:
        return 0.25
    if hour < 17:
        return 0.5
    if hour < 21:
        return 0.75
    return 1

#построим фичи из даты
df['weekday'] = df['pass_dttm'].apply(lambda x : 0 if x.weekday() < 5 else 1) 
df['pass_time'] = df['pass_dttm'].apply(lambda x : hour_period(x.hour)) 
df.drop(columns=['pass_dttm'], inplace=True)

In [69]:
#очевидно
df.drop(columns=['ticket_id','entrance_id', 'entrance_nm', 'id', 'line_nm'], inplace=True)

In [70]:
metro.head(3)

,Unnamed: 0,name,lat,lng
0,0,Новокосино,55.745113,37.864052
1,1,Новогиреево,55.752237,37.814587
2,2,Перово,55.750980,37.784220


In [71]:
stations = {}
for i in range(len(metro)):
    stations[metro.loc[i]['name'].replace('ё','е').lower()] = [metro.loc[i]['lat'], metro.loc[i]['lng']]
    
station_ids = {}
for i in range(len(kk)):
    station_ids[kk.iloc[i]['station_id']] = kk.iloc[i]['station_nm']

In [72]:
df['start_lat'] = df.station_nm.apply(lambda x : stations[x.replace('ё','е').lower()][0])
df['start_lng'] = df.station_nm.apply(lambda x : stations[x.replace('ё','е').lower()][1])
df['label_nm'] = df.label.apply(lambda x : station_ids[x])
df['end_lat'] = df.label_nm.apply(lambda x : stations[x.replace('ё','е').lower()][0])
df['end_lng'] = df.label_nm.apply(lambda x : stations[x.replace('ё','е').lower()][1])

In [73]:
df.head(5)

,ticket_type_nm,station_id,station_nm,line_id,time_to_under,label,weekday,pass_time,start_lat,start_lng,label_nm,end_lat,end_lng
0,Пропуск FacePay,11007,Лефортово,11,216.316667,8001,0,0.25,55.764444,37.702778,Авиамоторная,55.751933,37.717444
1,СК учащегося 30 дней,2006,Войковская,2,648.183333,9011,0,0.25,55.818923,37.497791,Нахимовский проспект,55.662379,37.605274
2,БСК дружинника г.Москвы,2006,Войковская,2,865.333333,7022,0,0.25,55.818923,37.497791,Улица 1905 года,55.763944,37.562271
3,30 дней,2006,Войковская,2,1048.233333,2022,0,0.25,55.818923,37.497791,Ховрино,55.877700,37.487700
4,КОШЕЛЕК,2006,Войковская,2,965.600000,2017,0,0.25,55.818923,37.497791,Речной вокзал,55.854152,37.476728


In [74]:
df.drop(columns=['station_nm', 'label_nm', 'line_id'], inplace=True)


In [75]:
df.head()

,ticket_type_nm,station_id,time_to_under,label,weekday,pass_time,start_lat,start_lng,end_lat,end_lng
0,Пропуск FacePay,11007,216.316667,8001,0,0.25,55.764444,37.702778,55.751933,37.717444
1,СК учащегося 30 дней,2006,648.183333,9011,0,0.25,55.818923,37.497791,55.662379,37.605274
2,БСК дружинника г.Москвы,2006,865.333333,7022,0,0.25,55.818923,37.497791,55.763944,37.562271
3,30 дней,2006,1048.233333,2022,0,0.25,55.818923,37.497791,55.877700,37.487700
4,КОШЕЛЕК,2006,965.600000,2017,0,0.25,55.818923,37.497791,55.854152,37.476728


In [76]:
ticket_types = df['ticket_type_nm'].unique()
# print(len(ticket_types))
# pd.get_dummies(df, columns=['ticket_type_nm'])

In [77]:
#опять какой-то мусор подсунули, много дубликатов
dict = {
    'КОШЕЛЕК': 'КОШЕЛЕК',
    'ББК': 'ББК',
    '30 дней СК студента': 'СК студента',
    '90 дней СК студента': 'СК студента',
    'СК студента 30 дней': 'СК студента',
    '90 дней СК аспиранта': 'СК студента',
    'СК аспиранта 90 дней': 'СК студента',
    'СК аспиранта 30 дней': 'СК студента',
    '30 дней СК аспиранта': 'СК студента',
    '30 дней СК ординатора': 'СК студента',
    '90 дней СК ординатора': 'СК студента',
    'СК студента 90 дней': 'СК студента',
    'СК ординатора 30 дней': 'СК студента',
    'СК ординатора 90 дней': 'СК студента',
    '30 дней СК ассистента-стажера': 'СК сотрудника',
    'СК ассистента-стажера 30 дней': 'СК сотрудника',
    'СК ассистента-стажера 90 дней': 'СК сотрудника',
    '30 дней СК учащегося': 'СК учащегося',
    'СК учащегося 90 дней': 'СК учащегося',
    '90 дней СК учащегося': 'СК учащегося',
    'СК учащегося 30 дней': 'СК учащегося',
    '30 дней ЕДИНЫЙ ТК': 'ТК',
    '30 дней': 'ТК',
    '90 дней ЕДИНЫЙ ТК': 'ТК',
    '365 дней ЕДИНЫЙ ТК': 'ТК',
    '365 дней': 'ТК',
    '90 дней': 'ТК',
    'Ультралайт Единый (70)': 'ТК',
    '30 дней  Пригород': 'ТК',
    '90 дней  Пригород': 'ТК',
    '60 поездок ЕДИНЫЙ ТК': 'ТК',
    '30 дней Пригород ТК': 'ТК',
    'ВЕСБ МОСКВА (7мин)': 'ТК',
    '1 сутки ЕДИНЫЙ ТК': 'ТК',
    '3 суток Единый ТК': 'ТК',
    '30 дней Пригород': 'ТК',
    '90 дней Пригород ТК': 'ТК',
    '1 сутки Пригород ТК': 'ТК',
    'Билет 1 сутки ЕДИНЫЙ': 'ТК',
    'Временный билет ММ': 'КОШЕЛЕК',
    '90 дней Пригород': 'ТК',
    '365 дней Пригород ТК': 'ТК',
    'Билет 3 суток ЕДИНЫЙ': 'ТК',
    'Социальная карта жителя Моск. области': 'СК',
    'Социальная карта москвича': 'СК',
    'ВЕСБ МО (тип 5)': 'ТК',
    '365 дней  Пригород': 'ТК',
    'ВЕСБ МО (7 мин)': 'ТК',
    'Социальная карта жителя Моск. области с сопровождающим': 'СК',
    'Социальная карта москвича с сопровождающим': 'СК',
    '365 дней Пригород': 'ТК',
    'ВЕСБ МО с сопровождающим': 'ТК',
    'ВЕСБ МОСКВА': 'ТК',
    'Безналичная транспортная карта': 'КОШЕЛЕК',
    '365 дней Пригород': 'ТК',
    'Пропуск сотрудника  метрополитена': 'СК сотрудника',
    'Пропуск сотрудника метрополитена': 'СК сотрудника',
    'Пропуск  сотрудника МЦК': 'СК сотрудника',
    'Пропуск сотрудника МЦК': 'СК сотрудника',
    'ВЛБ МОСКВА': 'ТК',
    'Пропуск руководителя метрополитена': 'СК сотрудника',
    'СК для сотрудника ГУВД г.Москвы': 'СК сотрудника',
    'БСК дружинника г.Москвы': 'СК сотрудника',
    'Пропуск сотрудника УВД по охране Метрополитена': 'СК сотрудника',
    'Пропуск FacePay': 'КОШЕЛЕК',
    'ВЕСБ МО (тип 6)': 'ТК'
}
df['ticket_type_nm'] = df['ticket_type_nm'].apply(lambda x: dict[x])

In [78]:
#что-то поменялось?
ticket_types = df['ticket_type_nm'].unique()
pd.DataFrame([ [x, df[df.ticket_type_nm==x].shape[0] ] for x in ticket_types], columns=['ticket_type_nm', 'count']).sort_values('count', ascending=False)

,ticket_type_nm,count
3,ТК,489289
0,КОШЕЛЕК,271952
6,СК,145089
4,СК студента,102781
1,СК учащегося,45958
5,ББК,29674
2,СК сотрудника,6278


In [79]:
enc = OrdinalEncoder()
df['ticket_type_nm'] = enc.fit_transform(df[['ticket_type_nm']]) 
Escaler = StandardScaler()
Escaler.fit(df[['ticket_type_nm']])
df['ticket_type_nm'] = Escaler.transform(df[['ticket_type_nm']])

# Dscaler = StandardScaler()
# Dscaler.fit(df[['pass_time']])
# df['pass_time'] = Dscaler.transform(df[['pass_time']])

In [80]:
df.head()

,ticket_type_nm,station_id,time_to_under,label,weekday,pass_time,start_lat,start_lng,end_lat,end_lng
0,-1.254947,11007,216.316667,8001,0,0.25,55.764444,37.702778,55.751933,37.717444
1,0.530976,2006,648.183333,9011,0,0.25,55.818923,37.497791,55.662379,37.605274
2,-0.361986,2006,865.333333,7022,0,0.25,55.818923,37.497791,55.763944,37.562271
3,0.977457,2006,1048.233333,2022,0,0.25,55.818923,37.497791,55.877700,37.487700
4,-1.254947,2006,965.600000,2017,0,0.25,55.818923,37.497791,55.854152,37.476728


In [81]:
#нормализуем координаты
Xscaler = StandardScaler()
Yscaler = StandardScaler()

X1scaler = StandardScaler()
Y1scaler = StandardScaler()

Xscaler.fit(df[['start_lat']])
Yscaler.fit(df[['start_lng']])

X1scaler.fit(df[['end_lat']])
Y1scaler.fit(df[['end_lng']])

df['start_lat'] = Xscaler.transform(df[['start_lat']])
df['start_lng'] = Yscaler.transform(df[['start_lng']])
df['end_lat'] = Xscaler.transform(df[['end_lat']])
df['end_lng'] = Yscaler.transform(df[['end_lng']])

/home/denis/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- end_lat
Feature names seen at fit time, yet now missing:
- start_lat

  warnings.warn(message, FutureWarning)
/home/denis/.local/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- end_lng
Feature names seen at fit time, yet now missing:
- start_lng

  warnings.warn(message, FutureWarning)


In [82]:
train, test = train_test_split(df, test_size=0.02, random_state=30)

In [83]:
df.head()

,ticket_type_nm,station_id,time_to_under,label,weekday,pass_time,start_lat,start_lng,end_lat,end_lng
0,-1.254947,11007,216.316667,8001,0,0.25,0.339782,0.373179,0.197129,0.452494
1,0.530976,2006,648.183333,9011,0,0.25,0.960962,-0.735402,-0.823982,-0.154128
2,-0.361986,2006,865.333333,7022,0,0.25,0.960962,-0.735402,0.334081,-0.386690
3,0.977457,2006,1048.233333,2022,0,0.25,0.960962,-0.735402,1.631148,-0.789974
4,-1.254947,2006,965.600000,2017,0,0.25,0.960962,-0.735402,1.362649,-0.849312


In [84]:
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.model_selection import GridSearchCV

features = ['start_lat', 'start_lng', 'pass_time', 'weekday', 'ticket_type_nm']

X_train = train[features]
y_train = train[['label']].values.ravel()

param_grid_knn = {
    'n_neighbors': [128],                                   
    'algorithm': ['kd_tree'],          
#     # 'metric': ['minkowski', 'manhattan', 'euclidean']
}

kNNModel_grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid_knn, verbose=3, cv=3, n_jobs=-1)
kNNModel_grid.fit(X_train, y_train)
# print(kNNModel_grid.best_estimator_)
# knn = KNeighborsClassifier(n_neighbors=128, algorithm='kd_tree')
# knn.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 3/3] END algorithm=kd_tree, n_neighbors=128;, score=0.057 total time=  51.1s
[CV 1/3] END algorithm=kd_tree, n_neighbors=128;, score=0.056 total time=  52.1s
[CV 2/3] END algorithm=kd_tree, n_neighbors=128;, score=0.057 total time=  52.3s


GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['kd_tree'], 'n_neighbors': [128]},
             verbose=3)

In [85]:
knn_predict = kNNModel_grid.predict(test[features])

In [86]:
recall_score(test['label'], knn_predict, average='micro')

0.06012556711424774